# The Pit Topography from Shadows (**PITS**) Tool
## Introduction
The Pit Topography from Shadows (**PITS**) tool is a dockerised Python framework which can automatically calculate an apparent depth (*h*) profile for Martian or Lunar pits from only one cropped, single- or multi-band, remote-sensing image. *h* is the relative depth of the pit between its rim and the edge of the shadow cast by the Sun - with the principle being that a deeper pit would cast a wider shadow. 

Pits, or pit craters, are near-circular depressions found in planetary surfaces which are generally formed through gravitational collapse. Pits will be primary targets for future space exploration and habitability, for their presence on most rocky Solar System surfaces and their potential to be entrances to sub-surface cavities.

PITS employs image segmentation in the form of *k*-means clustering with silhouette analysis to automatically detect the shadow and measure its width, wherby a *h* profile can be calculated. In addition to the *h* profile, **PITS** saves the extents of the detected shadow as a geo-referenced ESRI shapefile for visualisation in GIS software. This can be used to enhance the contrast of the pixels within the shadow to search for any deeper-shaded regions - possibly due to a cave entrance. **PITS** currently works with Mars Reconnaissance Orbiter (MRO) High Resolution Science Imaging Experiment (HiRISE) and Lunar Reconnaissance Orbiter (LRO) Narrow Angle Camera (NAC) imagery of Mars and the Moon, respectively.

Go to **PITS'** [GitHub](https://github.com/dlecorre387/Pit-Topography-from-Shadows/) repository to learn more about how to install the tool, how it works, and its accuracy when tested on MRO HiRISE imagery of Martian pits.

## PITS - Jupyter Notebook Tutorial
Typically, **PITS** is run by executing a Python script, called [`run_PITS.py`](run_PITS.py), within the terminal of a docker container. This script imports the functions from an additional script, called [`PITS_functions.py`](PITS_functions.py), and performs them in the correct order to get the desired outputs. [`PITS_plotter.py`](PITS_plotter.py) is also used for plotting the *h* profiles (since they are normally saved as CSV files) for the entire imagery dataset provided to **PITS**.

This Jupyter notebook tutorial has also been compiled in order to better explain the individual elements of the **PITS** algorithm, and how they have been implemented into Python. These steps (that would normally form one large 'for' loop) have been broken down into individual cells such that they can be explained, run in succession, and the result can be viewed afterwards. Consequently, this notebook imports functions from [`PITS_functions_tutorial.py`](PITS_functions_tutorial.py), which have been tailored specifically for this tutorial. 

In this tutorial, we will be applying the **PITS** tool to the MRO HiRISE image ESP_033342_1660, which contains a Martian pit. Pit location labels (in ESRI shapefile format) have already been provided for cropping this HiRISE image product down to the extents of the pit. Labels of the pit's shadow have also been produced such that the accuracy of **PITS'** automated shadow extraction can be tested. Red-band and colour versions of ESP_033342_1660 are also available, showing the ability of **PITS** to produce *h* profiles for single and multi-band imagery.

### Importing Packages
The cell below imports all necessary python packages and the functions from the [`PITS_functions_tutorial.py`](PITS_functions_tutorial.py) file. It also defines the logger which prints various info/error messages where applicable.

In [ ]:
# Import modules
import os
import shutil
import logging
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import mode

# Import PITS functions
from PITS_functions_tutorial import *

# Configure logger
logging.basicConfig(level = logging.INFO,
                    format='| %(asctime)s | %(levelname)s | Message: %(message)s',
                    datefmt='%d/%m/%y @ %H:%M:%S')

### Defining Directory to Input Data
In the cell below, the full path to the directory containing the necessary input data to allow **PITS** to run needs to be defined. When running the docker container for the first time, you can either copy the data to a location in the container, mount a local volume to a location in the container, or point to it directly. In this directory, there should be four folders to start with:
1. `input` - Contains the input imagery that **PITS** is meant to derived *h* profiles from,
2. `labels` - Contains the ESRI shapefile labels of the pit's location (as rectangular polygons) in the image,
3. `metadata` - Contains the cumulative PDS3 index file with all the necessary metadata for your images,
4. `testing` - Contains the manual ESRI shapefile labels of the pit's shadows for testing shadow extraction performance.

When running **PITS** for the first time, a fifth folder will be created, called `output`, which will house all of the outputs of the tool - such as *h* profiles, shapefiles of the detected shadows, and a CSV of all results.

In [ ]:
# Define the full path to the directory containing the necessary input data (ending with a /)
data_dir = '/data/'

# Check that the data directory exists
if not os.path.exists(data_dir):
    raise OSError(f"Directory '{data_dir}' could not be found or does not exist.")

# Check that the folders input, labels, metadata and testing are present
folders = ['input/', 'labels/', 'metadata/', 'testing/']
for folder in folders:
    if os.path.exists(data_dir) and not os.path.exists(os.path.join(data_dir, folder)):
        raise OSError(f"Directory '{os.path.join(data_dir, folder)}' could not be found or does not exist.")
    
# Define the full paths to the individual input folders
input_dir = os.path.join(data_dir, 'input/')
metadata_dir = os.path.join(data_dir, 'metadata/')
labels_dir = os.path.join(data_dir, 'labels/')
testing_dir = os.path.join(data_dir, 'testing/')
output_dir = os.path.join(data_dir, 'output/')

### Input Parameters
The following cell is where the user needs to define a number of the input parameters for the PITS tool to operate. When running **PITS** in the command line, a number of mandatory and optional arguments can be passed. For this Jupyter notebook to work, these command line arguments are given as variables which the user will need to define. We also add an additional optional argument called `colour`, which will tell **PITS** to run on the red, colour or both versions of ESP_033342_1660.

In [ ]:
'''
Required Parameters:

dataset:    The name of the dataset whose images will be used to calculate apparent depths.
            Currently supported options are "hirise-rdr" (for MRO HiRISE RDR version 1.1 
            images of Mars) and "lronac-edr" (for LROC NAC EDR images of the Moon). This 
            is required since there is a different process for retrieving sensing 
            information for each dataset.
            (No default / Type: str)

cropping:   Crop each larger input image to the extents of the pit feature using user-provided
            ESRI shapefile rectangular labels of the pit's location. These shapefiles must 
            include or be equal to the full product name of the corresponding image file, e.g. 
            label_ESP_033342_1660_RED.shp for the HiRISE image ESP_033342_1660_RED.JP2.
            (No default / Type: bool)
'''

# Required parameters
dataset = None
cropping = None

# Checking the required parameters are the correct types
if dataset != 'hirise-rdr' and dataset != 'lronac-edr' and dataset != None:
    raise ValueError("Supported options for dataset are 'hirise-rdr' for HiRISE RDRV11 data and 'lronac-edr' for LRO NAC EDR data.")
elif dataset == None:
    raise ValueError("Please define the variable 'dataset'. Supported options for dataset are 'hirise-rdr' for HiRISE RDRV11 data and 'lronac-edr' for LRO NAC EDR data.")
if cropping != True and cropping != False and cropping != None:
    raise ValueError("Supported options for 'cropping' are True or False.")
elif cropping == None:
    raise ValueError("Please define the variable 'cropping'. Supported options for 'cropping' are True or False.")

'''
Optional Parameters:

colour:     Apply PITS to the red, colour or both versions of the image. Set 'colour' to 'red', 
            'colour' or 'both', respectively.
            (Default: 'both' / Type: str)

testing:    Calculate the precision, recall and F1 score of shadow pixel detections in each image 
            using user-provided ESRI shapefile labels of the pit's shadow. 
            (Default: True / Type: bool)
            
factor:     The factor by which the cropped input image and labels will be down-scaled when
            calculating the silhouette coefficients during shadow extraction. 
            (Default: 10 / Type: float)
'''

# Optional parameters
colour = 'both'
testing = True
factor = 10

# Checking that 'colour' is one of the options
if colour not in ['red', 'colour', 'both']:
    raise ValueError("Variable 'colour' must be equal to 'red', 'colour' or 'both'.")

# Check that 'testing' is a boolean
if not isinstance(testing, bool):
    raise ValueError("Variable 'testing' must be a boolean.")

# Checking that 'factor' is a float and more than one
if not isinstance(factor, float) and not isinstance(factor, int):
    raise ValueError("Variable 'factor' must be a int/float.")
if factor < 1:
    raise ValueError("Variable 'factor' must be larger than 1.")

# Do not change these variables
cluster_range = np.arange(4, 14, 1)
miss_rates = [0.004280421, 0.00611175]
FD_rates = [0.052279632, 0.059128667]

### Creating and Cleaning Output Directory
This cell creates the directory where **PITS'** outputs will be stored, or cleans it if it already exists.

In [ ]:
# Clean or create output folder
if os.path.exists(output_dir):
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file, e))
    logging.info(f"Output folder '{output_dir}' cleaned.")
elif not os.path.exists(output_dir):
    os.makedirs(output_dir)
    logging.info(f"Output folder '{output_dir}' created.")
   
# Create sub-folders for separating, h profiles, shadow shapefiles and other results
profiles_dir = os.path.join(output_dir, 'profiles/')
shadows_dir = os.path.join(output_dir, 'shadows/')
subfolders = [profiles_dir, shadows_dir]
for subfolder in subfolders:
    if not os.path.exists(subfolder):
        try:
            os.makedirs(subfolder)
        except Exception as e:
            print('Failed to create subfolder %s. Reason %s' % (subfolder, e))
logging.info(f"Sub-folders created.")

### Opening Arrays for Storing Results
Since we know the number of images in the input directory that **PITS** is to be applied to, we can open some empty arrays to save run-time when storing information (to be saved as a CSV later).

In [ ]:
# List all filenames to be analysed without XML or TXT files
filenames = [file for file in os.listdir(input_dir) if not file.endswith('.xml') if not file.endswith('.txt')]

# Filter the list of filenames depending on if PITS will be applied to red and/or colour images
if colour == 'red':
    filenames = [filename for filename in filenames if 'RED' in filename]
elif colour == 'colour':
    filenames = [filename for filename in filenames if 'COLOR' in filename]

logging.info(f"The following files will be analysed ({len(filenames)} in total):")
for file_n, filename in enumerate(filenames):
    print(f"File {file_n+1}:     {filename}")

# Open empty arrays to store sensing info
resolutions = np.empty((len(filenames)))
inc_angles = np.empty((len(filenames)))
solar_azim_angles = np.empty((len(filenames)))
sc_azim_angles = np.empty((len(filenames)))
em_angles = np.empty((len(filenames)))
delta_em_angles = np.empty((len(filenames)))
em_angle_pars = np.empty((len(filenames)))
em_angle_perps = np.empty((len(filenames)))
phase_angles = np.empty((len(filenames)))

# Open empty arrays to store the centre and maximum apparent depths (h)
centre_hs = np.empty((len(filenames)))
pos_centre_hs = np.empty((len(filenames)))
neg_centre_hs = np.empty((len(filenames)))
max_hs = np.empty((len(filenames)))
pos_max_hs = np.empty((len(filenames)))
neg_max_hs = np.empty((len(filenames)))

# Set up arrays for storing testing metrics
if testing:
    P_shadow = np.empty(len(filenames))
    R_shadow = np.empty(len(filenames))
    F1_shadow = np.empty(len(filenames))

# Store the silhouette coefficients/scores
silhouettes = np.empty((len(filenames), len(cluster_range)))

### Cropping Larger Image Products and Reading Image Metadata
In this cell, all larger image products are cropped using the pit location shapefile labels. It will also read in all the relevant sensing information and sensor data from the PDS3 cumulative index file which should be placed in the metadata directory before building the docker image. These files can be retrieved from NASA's Planetary Data System ([PDS](https://pds.nasa.gov/)) and filtered to the relevant images using the BASH script [`filter_index_files.sh`](filter_index_files.sh) provided along with **PITS**. This greatly improves the run-time as without this, **PITS** has to look through several thousand lines of a text file to find the correct row of metadata.

In [ ]:
# Open progress bar
pbar = tqdm(total=len(filenames), desc='Progress')

# Loop through each image
for i, filename in enumerate(filenames):
        
    # Initialise DataPreparer class
    DataPrep = DataPreparer(filename=filename,
                            input_dir=input_dir, 
                            metadata_dir=metadata_dir,
                            labels_dir=labels_dir,
                            testing_dir=testing_dir,
                            output_dir=output_dir)
    
    if cropping:
        
        # Crop the image using provided shapefile labels and return raster information [resolution in m, lat/lon in deg]
        cropped_image, resolution, min_longitude, max_longitude, min_latitude, max_latitude, geotransform, projection, n_bands, x_size, y_size = DataPrep.crop_image()

        # Retrieve sensing angles at time of acquisition [in radians]
        inc_angle, solar_azim_angle, sc_azim_angle, phase_angle, em_angle, delta_em_angle, em_angle_par, em_angle_perp = DataPrep.read_metadata(dataset, min_longitude, max_longitude, min_latitude, max_latitude)
        
    elif not cropping:
        
        # Retrieve sensing angles at time of acquisition [in radians]
        inc_angle, solar_azim_angle, sc_azim_angle, phase_angle, em_angle, delta_em_angle, em_angle_par, em_angle_perp = DataPrep.read_metadata(dataset, None, None)
    
    # Store metadata in arrays for saving later
    resolutions[i], inc_angles[i], solar_azim_angles[i], sc_azim_angles[i], phase_angles[i], em_angles[i], delta_em_angles[i], em_angle_pars[i], em_angle_perps[i] = resolution, inc_angle, solar_azim_angle, sc_azim_angle, phase_angle, em_angle, delta_em_angle, em_angle_par, em_angle_perp

    # Update the progress bar
    pbar.update(1)
pbar.close()

if cropping:
    logging.info("All images cropped and metadata acquired.")
elif not cropping:
    logging.info("All metadata acquired.")

### Automated Shadow Extraction
**PITS** employs *k*-means clustering with silhouette analysis to segment each image into *k* number of clusters. This means that each individual pixel in the input image is assigned a cluster label from 0 to *k* - 1. If **PITS** is applied to multi-band images, each band is clustered separately and the modal cluster label for each pixel across all bands is then assigned. The darkest cluster (as it appears in the input image) is then extracted and classified as the shadow. Silhouette analysis is used to automatically select the value for *k* which returns the darkest cluster which is the most consistent within itself, and  the most distinct from all other clusters. This results in a binary mask, where shadow and background pixels are assigned a 1 and 0, respectively.

Some post-processing is then performed on this binary shadow mask, as to ensure a correct shadow width measurement later on. Firstly, the largest continuous shadow is separated, with all others being removed from the mask (i.e. assigned to background). This is to filter out any small shadows cast by features such as boulders rather than the shadow's rim. Then in order to remove any noisy bright pixels from the shadow, **PITS** performs morphological closing on all holes encased within the shadow mask with an area less than 10 pixels. Any remaining holes in the shadow are expected to be bright features (again, such as boulders) which are protruding above the shadow. Shadow pixels cast by these features are removed later on in the algorithm, as to not over estimate the depth of the pit at these points.

In [ ]:
# Re-define the input directory if images were cropped earlier
if cropping:
    input_dir = os.path.join(output_dir, 'cropped/')

# List all filenames to be analysed without .XML files
filenames = [file for file in os.listdir(input_dir) if not file.endswith('.xml')]

# Filter the list of filenames depending on if PITS will be applied to red and/or colour images
if colour == 'red':
    filenames = [filename for filename in filenames if 'RED' in filename]
elif colour == 'colour':
    filenames = [filename for filename in filenames if 'COLOR' in filename]

# Open progress bar
pbar = tqdm(total=len(filenames)*cluster_range.size, desc='Progress')

# Loop through each image
for i, filename in enumerate(filenames):
    
    # Get the product identification of the image from the filename
    name = os.path.splitext(filename)[0]

    # Initialise DataPreparer class
    DataPrep = DataPreparer(filename=filename,
                            input_dir=input_dir, 
                            metadata_dir=metadata_dir,
                            labels_dir=labels_dir,
                            testing_dir=testing_dir,
                            output_dir=output_dir)

    # Read the cropped image and return raster information [resolution in m]
    cropped_image, _, _, _, n_bands, x_size, y_size = DataPrep.read_cropped_im()

    # Open an empty array to store the all the sorted labels for each value of k   
    all_sorted_labels = np.empty((len(cluster_range), x_size, y_size))

    # Loop over different numbers of kmeans clusters and shadow cluster threshold
    for c, n_clusters in enumerate(cluster_range):

        # Cluster the image and sort the labels for a single-band image
        if n_bands == 1:
            
            # Initialise the ShadowExtractor class
            ShadExt = ShadowExtractor(cropped_image=cropped_image,
                                    n_clusters=n_clusters,
                                    factor=factor)
            
            # Cluster image so that each pixel is assigned a label
            labels = ShadExt.kmeans_clustering()
            
            # Sort the labels by brightness
            sorted_labels = ShadExt.sort_clusters(labels)
            
            # Calculate and save the average silhouette coefficient for the darkest cluster
            silhouettes[i, c] = ShadExt.calc_silh_coefficient(sorted_labels)
                
        # Cluster and sort the labels of each band in a multi-band image
        elif n_bands > 1:
            
            # Store the silhouette coefficient/score and labels for each band
            band_silhouettes = np.empty((n_bands))
            band_labels = np.empty((n_bands, x_size, y_size))
            
            # Loop through each band
            for band in np.arange(n_bands):
                
                # Initialise the ShadowExtractor class
                ShadExt = ShadowExtractor(cropped_image=cropped_image[band, :, :],
                                        n_clusters=n_clusters,
                                        factor=factor)
                
                # Cluster the individual band so that each pixel is assigned a label
                labels = ShadExt.kmeans_clustering()
                
                # Sort the labels for this band by brightness
                band_labels[band, :, :] = ShadExt.sort_clusters(labels)
                
                # Calculate and save the average silhouette coefficient for the darkest cluster
                band_silhouettes[band] = ShadExt.calc_silh_coefficient(band_labels[band, :, :])
            
            # Average the silhouette coefficients/scores across the bands
            silhouettes[i, c] = np.mean(band_silhouettes)
                
            # Calculate modal labels if applied to colour images
            sorted_labels = (mode(band_labels, axis=0).mode).astype(int)
        
        else:
            raise ValueError("Number of bands should not be zero.")
        
        # Store the sorted labels for later and save the number of iterations for this value of k
        all_sorted_labels[c, :, :] = sorted_labels

        # Update the progress bar
        pbar.update(1)

    # Find the labels for k which gave the highest silhouette coefficient/score
    ind = int(np.argmax(silhouettes[i, :]))

    # Use the labels which maximised the darkest clusters silhouette coefficient
    raw_shadow = np.where(all_sorted_labels[ind, :, :] == np.amin(all_sorted_labels[ind, :, :]), 1, 0)
    
    # Initialise the PostProcessor class
    PostProc = PostProcessor(shadow=raw_shadow)
    
    # Remove all small shadow detections so only the main shadow remains, then detect and fill bright features in shadow mask
    main_shadow, filled_shadow = PostProc.post_processing()

    # Compare the detected shadows to the manually-labelled shadow shapefiles to get testing metrics
    if testing:
    
        # Read in ground truth if testing
        true_shadow, _ = DataPrep.read_ground_truth(n_bands, cropped_image, geotransform, projection)
                
        # Initialise ShadowTester class
        ShadTest = ShadowTester(main_shadow=main_shadow,
                                true_shadow=true_shadow)
    
        # Calculate and store the precision, recall and F1 scores
        P_shadow[i], R_shadow[i], F1_shadow[i] = ShadTest.calc_shadow_metrics()
        
        # Plot the detected shadow and colour-code it (R: FP, G: TP, B: FN)
        non_shadow = np.where(true_shadow == 0, 1, 0)
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
        ax1.imshow(true_shadow, cmap='gray', interpolation='none')
        ax1.axis('off')
        ax1.set_title(f"Shadow 'Ground Truth' for\n{name}")
        ax2.imshow(main_shadow, cmap='gray', interpolation='none')
        ax2.axis('off')
        ax2.set_title(f"Extracted Shadow for\n{name}")
        colour_array = np.empty((x_size, y_size, 3))
        colour_array[:, :, 0] = np.where(main_shadow * non_shadow == 1, 1, 0)
        colour_array[:, :, 1] = np.where(main_shadow * true_shadow == 1, 1, 0)
        colour_array[:, :, 2] = np.where(true_shadow - main_shadow == 1, 1, 0)
        mask = np.sum(colour_array, axis=2) == 0
        colour_array[mask, :] = 1
        ax3.imshow(colour_array, interpolation='none')
        ax3.axis('off')
        ax3.set_title(f"Shadow Extraction Performance for\n{name}")
        plt.show()

    # Save the shadow as a geo-referenced shapefile
    DataPrep.save_shadow(main_shadow, filled_shadow, geotransform, projection)

    # Plot the input image and detected shadow
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
    if n_bands == 1:
        ax1.imshow(cropped_image, cmap='gray', aspect='equal', interpolation='none')
        ax1.set_title(f"{name}")
    elif n_bands > 1:
        ax1.imshow(cropped_image[0, :, :], cmap='gray', aspect='equal', interpolation='none')
        ax1.set_title(f"{name} (Red Band Only)")
    ax1.axis('off')
    ax2.imshow(all_sorted_labels[ind, :, :], aspect='equal', interpolation='none')
    ax2.axis('off')
    ax2.set_title(r"$k$-means Clusters")
    ax3.imshow(main_shadow, cmap='gray', aspect='equal', interpolation='none')
    ax3.axis('off')
    ax3.set_title("Detected Main Shadow")
    plt.show()
    
pbar.close()

if testing:
    logging.info(f"Shadows automatically extracted from all input images, processed, tested and saved to {shadows_dir}")

elif not testing:
    logging.info(f"Shadows automatically extracted from all input images, processed and saved to {shadows_dir}")

### Calculate Apparent Depth (*h*) Profiles
**PITS** rotates the binary shadow mask by the Sun's azimuth angle relative to North ($\varphi$). It can then measure the width of the shadow along the Sun's line of sight as observed by the satellite (*S<sub>obs</sub>*) at each pixel in the shadows length. *S<sub>obs</sub>* is then corrected for non-nadir observations to obtain the true shadow width (*S<sub>true</sub>*) as if the satellite was pointing straight downwards at the surface. *h* is then derived from these *S<sub>true</sub>* measurements by considering the incidence angle of the Sun ($\alpha$) for this particular image.

In [ ]:
# Open progress bar
pbar = tqdm(total=len(filenames), desc='Progress')

# Loop through each image
for i, filename in enumerate(filenames):
    
    # Get the product identification of the image from the filename
    name = os.path.splitext(filename)[0]

    # Recalculate miss and false discovery rates if testing
    if testing:    
        
        # Use the errors (miss rate and FD rate) calculated by comparing to the ground truth
        miss_rate = 1 - R_shadow[i]
        FD_rate = 1 - P_shadow[i]
    
    # Don't recalculate miss and false discovery rates if no testing
    elif not testing:
        
        # Retrieve the correct miss and false discovery rates for the number of bands
        if n_bands == 1:
            miss_rate, FD_rate = miss_rates[0], FD_rates[0]
        elif n_bands > 1:
            miss_rate, FD_rate = miss_rates[1], FD_rates[1]
        else:
            raise ValueError("Number of bands should not be zero.")

    # Initialise DataPreparer class
    DataPrep = DataPreparer(filename=filename,
                            input_dir=input_dir, 
                            metadata_dir=metadata_dir,
                            labels_dir=labels_dir,
                            testing_dir=testing_dir,
                            output_dir=output_dir)
    
    # Read in the main shadow
    main_shadow = DataPrep.read_shadow('main_shadow')
    filled_shadow = DataPrep.read_shadow('filled_shadow')
    
    # If no bright features were found within the shadow mask
    if filled_shadow is None:
    
        # Initialise the DepthCalculator class
        DepCalc = DepthCalculator(shadow_list=[main_shadow],
                                resolution=resolutions[i],
                                inc_angle=inc_angles[i],
                                em_angle=em_angles[i],
                                em_angle_par=em_angle_pars[i],
                                em_angle_perp=em_angle_perps[i],
                                solar_azim_angle=solar_azim_angles[i],
                                phase_angle=phase_angles[i])
        
        # Align the main shadow to the Sun's line of sight
        aligned_shadow = DepCalc.align_shadow()
        
        # Measure the observed shadow widths of the aligned shadow [in m]
        S_obs, coords, edge, rim = DepCalc.measure_shadow(aligned_shadow)
        S_obs = S_obs[S_obs != 0]
        
        # Calculate the upper and lower bounds of the uncertainty in the observed shadow width [in m]
        pos_delta_S_obs = miss_rate * S_obs
        neg_delta_S_obs = FD_rate * S_obs
        
    # If there were bright features found within the shadow mask
    elif filled_shadow is not None:
        
        # Initialise the DepthCalculator class
        DepCalc = DepthCalculator(shadow_list=[main_shadow, filled_shadow],
                                resolution=resolutions[i],
                                inc_angle=inc_angles[i],
                                em_angle=em_angles[i],
                                em_angle_par=em_angle_pars[i],
                                em_angle_perp=em_angle_perps[i],
                                solar_azim_angle=solar_azim_angles[i],
                                phase_angle=phase_angles[i])
        
        # Align the main (non-filled) and filled shadows to the Sun's line of sight
        aligned_main_shadow, aligned_filled_shadow = DepCalc.align_shadow()
        
        # Filter out all shadow pixels which may be caused by bright features within the main shadow
        aligned_filtered_shadow = DepCalc.remove_bright_features(aligned_main_shadow, aligned_filled_shadow)
        
        # Measure the observed shadow widths of the filtered shadow [in m]
        S_obs_filtered, coords_filtered, edge_filtered, rim_filtered = DepCalc.measure_shadow(aligned_filtered_shadow)
        
        # Measure the observed shadow widths of the filled shadow [in m]
        S_obs_filled, coords_filled, edge_filled, rim_filled = DepCalc.measure_shadow(aligned_filled_shadow)
        
        # Find and remove elements where both of the filtered and filled observed width measurements are zero
        zero_filter = np.logical_and(S_obs_filtered != 0, S_obs_filled != 0)
        S_obs_filtered = S_obs_filtered[zero_filter]
        S_obs_filled = S_obs_filled[zero_filter]
        
        # Calculate the average of the filled and filtered observed shadow widths [in m]
        S_obs = (S_obs_filled + S_obs_filtered) / 2
        
        # Calculate the upper and lower bounds of the averaged observed shadow width [in m]
        pos_delta_S_obs = np.maximum(S_obs_filled, S_obs_filtered) - S_obs + (S_obs * miss_rate)
        neg_delta_S_obs = S_obs - np.minimum(S_obs_filled, S_obs_filtered) + (S_obs * FD_rate)                
    
    # Calculate the observed apparent depth before correcting the shadow width [in m]
    h_obs = DepCalc.calculate_h(S_obs)
    
    # Calculate the observed length of the shadow [in m]
    L_obs = resolution * np.arange(0, S_obs.size)
    
    # Find the true shadow width and length by correcting for the satellite emission angle [in m]
    S_true, L_true = DepCalc.correct_shadow_width(S_obs, L_obs)
    
    # Calculate the true apparent depth now that the width has been corrected [in m]
    h_true = DepCalc.calculate_h(S_true)
    
    # Propagate the uncertainties in S_obs and the emission angle to h_true
    pos_delta_h_obs, neg_delta_h_obs, pos_delta_h_true, neg_delta_h_true = DepCalc.propagate_uncertainties(S_obs, pos_delta_S_obs, neg_delta_S_obs, delta_em_angles[i])
    
    # Save the apparent depth profile as a CSV file for plotting later
    DataPrep.save_h_profile(L_obs, h_obs, pos_delta_h_obs, neg_delta_h_obs, L_true, h_true, pos_delta_h_true, neg_delta_h_true)
        
    # Find the centre apparent depths to add to the results table
    ind = int(h_true.size / 2)
    centre_hs[i], pos_centre_hs[i], neg_centre_hs[i] = h_true[ind], pos_delta_h_true[ind], neg_delta_h_true[ind]
    
    # Find the maximum apparent depths to add to the results table
    max_hs[i], pos_max_hs[i], neg_max_hs[i] = np.amax(h_true), np.amax(pos_delta_h_true), np.amax(neg_delta_h_true)
    
    # Plot the apparent depth profile
    max_depth = np.amax(h_true) + pos_delta_h_true[np.argmax(h_true)]
    fig, ax = plt.subplots(figsize=(5,5))
    ax.set_aspect('equal')
    ax.plot(L_obs, h_obs, 'r--', alpha=0.5, label=r'$h_{obs}$')
    ax.fill_between(L_obs, h_obs - neg_delta_h_obs, h_obs + pos_delta_h_obs, alpha=0.1, color='red', label=r'$\Delta h_{obs}$')
    ax.plot(L_true, h_true, color='green', label=r'$h$')
    ax.fill_between(L_true, h_true - neg_delta_h_true, h_true + pos_delta_h_true, alpha=0.4, color='green', label=r'$\Delta h$')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=4, frameon=False)
    ax.set_xlim(L_true[0], L_true[-1])
    ax.set_ylim(0, np.ceil(max_depth / 10)*10)
    ax.set_xlabel(r"Shadow length ($L$) [m]")
    ax.set_ylabel(r"Apparent depth ($h$) [m]")
    ax.set_title("Apparent Depth Profile\nfor {}".format(name))
    ax.invert_yaxis()
    plt.show()

    # Update the progress bar
    pbar.update(1)
pbar.close()

## Saving Results
Lastly, PITS saves the results and corresponding sensing information for each image to a CSV file. If the automatically detected shadows have also been compared with manually produced labels, then the performance metrics are also saved to a separate CSV file.

In [ ]:
# Save testing performances
if testing:

    logging.info("Shadow extraction performance metrics:")
    logging.info("Average miss rate: {}".format(1 - np.mean(R_shadow)))
    logging.info("Average false discovery rate: {}".format(1 - np.mean(P_shadow)))
    logging.info("Average F1 score: {}".format(np.mean(F1_shadow)))

    # Store testing results in a structured array
    dt = np.dtype([('i', 'U32'), ('P_shadow', float), ('R_shadow', float), ('F1_shadow', float)])
    array = np.empty(len(filenames), dtype=dt)

    # Store the filenames, and the corresponding testing metrics
    array['i'] = [os.path.splitext(filename)[0] for filename in filenames]
    array['P_shadow'] = P_shadow
    array['R_shadow'] = R_shadow
    array['F1_shadow'] = F1_shadow

    # Save to a csv file with appropriate headers for reference
    np.savetxt(os.path.join(output_dir, 'PITS_testing.csv'), 
            array,
            delimiter=',',
            fmt='%s, %f, %f, %f',
            header='Image Name, Precision, Recall, F1 Score')
    
    logging.info("Shade extraction performance tested for all {} images and outputs saved to {}".format(len(filenames), output_dir))

# Store results and image information in a structured array
dt = np.dtype([('filename', 'U32'),
                ('res', float), ('inc', float), ('s_azim', float), ('em', float), ('sc_azim', float),
                ('h_c', float), ('pos_h_c', float), ('neg_h_c', float),
                ('h_m', float), ('pos_h_m', float), ('neg_h_m', float)])
array = np.empty(len(filenames), dtype=dt)

# Store filenames and sensing information
array['filename'] = [os.path.splitext(filename)[0] for filename in filenames]
array['res'] = resolutions * (180 / np.pi)
array['inc'] = inc_angles * (180 / np.pi)
array['s_azim'] = solar_azim_angles * (180 / np.pi)
array['em'] = em_angles * (180 / np.pi)
array['sc_azim'] = sc_azim_angles * (180 / np.pi)

# Store the corrected apparent depths (and uncertainties) at the shadow's centre
array['h_c'] = centre_hs
array['pos_h_c'] = pos_centre_hs
array['neg_h_c'] = neg_centre_hs

# Store the corrected maximum apparent depths (and uncertainties)
array['h_m'] = max_hs
array['pos_h_m'] = pos_max_hs
array['neg_h_m'] = neg_max_hs

# Save to a csv file with appropriate headers for reference
np.savetxt(os.path.join(output_dir, 'PITS_results.csv'), 
        array,
        delimiter=',',
        fmt='%s, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f',
        header='Image Name, Resolution [m], Incidence Angle [deg], Solar Azimuth Angle [deg], Emission Angle [deg], Spacecraft Azimuth Angle [deg], Centre h [m], +, -, Maximum h [m], +, -')

logging.info("All {} images analysed and outputs saved to {}".format(len(filenames), output_dir))

## Acknowledgements
This project is part of the Europlanet 2024 RI which has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149.